<a href="https://colab.research.google.com/github/jianzhichun/8009/blob/main/group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)

1.11.0+cu102


In [38]:
# !pip uninstall torch-scatter
# !pip uninstall torch-sparse
!pip install -q torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu102.html

#if you are doing it on your local machine, pls figure out your pytorch version and the cuda version and modify the following code accordingly.
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu102.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu102.html
!pip install -q networkx

In [31]:
import torch
import os
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt

To create the dataset we need to convert the raw information into a ```Data``` object (a graph) in PyG.

The first step is to load the csv files, this can be done manually or using some data library as Pandas:

### Open Graph Benchmark datasets

Open Graph Benchmark is available as a python library, to install it just run

```pip install ogb```

OGB allows to load a dataset in three ways: for PyG applications, for DGL (Deep Graph Library, another widely used tool for GNNs in python) and in an 'agnostic' manner. There is a naming convention to load a dataset, depending on the task an the dataset name:

    ogbn-[name]: for node tasks
    ogbg-[name]: for graph tasks
    ogbl-[name]: for link tasks

In [4]:
!pip install ogb

     |████████████████████████████████| 78 kB 3.9 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=a0b209b7a6648736c3c933a2103071f34296c3ff20e44357f1f64eb04a8d8d94
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils


In [32]:
from ogb.nodeproppred import PygNodePropPredDataset
#from ogb.graphproppred import PygGraphPropPredDataset
#from ogb.linkproppred import PygLinkPropPredDataset

In [50]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name = dataset_name, root='data', transform=T.ToSparseTensor()) 

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph = dataset[0] 

### You have to write a short proposal for me to review.

In this project, I will try to use Graph Neural Networks to predict a collection of papers from arXiv. We will build our first graph neural network using PyTorch Geometric and apply it to node attribute prediction (node classification). And We will build a graph neural network using the GCN operator.

### Describe the problem and the dataset clearly.

#### What’s the input, what’s the output?

In [34]:
print(f'Input: the graph data with {graph.num_features} features')
print(f'Output: the accuracy of our prediction model')

Input: the graph data with 128 features
Output: the accuracy of our prediction model


#### Why is the problem important?

Finding an accurate model can help us classify papers and predict the properties of papers.

#### What’s the most popular baseline method?

GCN, Node2Vec, MLP...

#### What’s the evaluation metric?

* The standardized evaluation protocol from ogb.
* The accuracy, precision, recall and F1-score

### Prediction Model

#### GCN

In [26]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        # TODO: Implement this function that initializes self.convs, 
        # self.bns, and self.softmax.

        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = torch.nn.ModuleList(
            [GCNConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GCNConv(in_channels=hidden_dim, out_channels=hidden_dim)                             
                for i in range(num_layers-2)] + 
            [GCNConv(in_channels=hidden_dim, out_channels=output_dim)]    
        )

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(num_features=hidden_dim) 
                for i in range(num_layers-1)
        ])
        

        # The log softmax layer
        self.softmax = torch.nn.LogSoftmax()

        ############# Your code here ############
        ## Note:
        ## 1. You should use torch.nn.ModuleList for self.convs and self.bns
        ## 2. self.convs has num_layers GCNConv layers
        ## 3. self.bns has num_layers - 1 BatchNorm1d layers
        ## 4. You should use torch.nn.LogSoftmax for self.softmax
        ## 5. The parameters you can set for GCNConv include 'in_channels' and 
        ## 'out_channels'. More information please refer to the documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## 6. The only parameter you need to set for BatchNorm1d is 'num_features'
        ## More information please refer to the documentation: 
        ## https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
        ## (~10 lines of code)

        #########################################

        # Probability of an element to be zeroed
        self.dropout = dropout

        # Skip classification layer and return node embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        # TODO: Implement this function that takes the feature tensor x,
        # edge_index tensor adj_t and returns the output tensor as
        # shown in the figure.
        for conv, bn in zip(self.convs[:-1], self.bns):
            x1 = F.relu(bn(conv(x, adj_t)))
            if self.training:
                x1 = F.dropout(x1, p=self.dropout)
            x = x1
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)

        ############# Your code here ############
        ## Note:
        ## 1. Construct the network as showing in the figure
        ## 2. torch.nn.functional.relu and torch.nn.functional.dropout are useful
        ## More information please refer to the documentation:
        ## https://pytorch.org/docs/stable/nn.functional.html
        ## 3. Don't forget to set F.dropout training to self.training
        ## 4. If return_embeds is True, then skip the last softmax layer
        ## (~7 lines of code)

        #########################################

        return out

#### GGNN

In [ ]:
# TODO

### Train & Evaluation

In [51]:
from ogb.nodeproppred import Evaluator
def train(model, data, train_idx, optimizer, loss_fn):
    # TODO: Implement this function that trains the model by 
    # using the given optimizer and loss_fn.
    model.train()
    loss = 0

    ############# Your code here ############
    ## Note:
    ## 1. Zero grad the optimizer
    ## 2. Feed the data into the model
    ## 3. Slicing the model output and label by train_idx
    ## 4. Feed the sliced output and label to loss_fn
    ## (~4 lines of code)
    optimizer.zero_grad()
    out = model(data.x, data.adj_t)
    loss = loss_fn(out[train_idx], data.y[train_idx].reshape(-1))

    #########################################

    loss.backward()
    optimizer.step()

    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    # TODO: Implement this function that tests the model by 
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None

    ############# Your code here ############
    ## (~1 line of code)
    ## Note:
    ## 1. No index slicing here
    out = model(data.x, data.adj_t)
    #########################################

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

### Apply & Compare

#### GCN

In [52]:
model = GCN(graph.num_features, 256, dataset.num_classes, 3, 0.5).to('cpu')
evaluator = Evaluator(name='ogbn-arxiv')
import copy

# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

# TODO
epochs = 100
for epoch in range(1, 1 + epochs):
  loss = train(model, graph, train_idx, optimizer, loss_fn)
  result = test(model, graph, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
    best_valid_acc = valid_acc
    best_model = copy.deepcopy(model)
best_result = test(best_model, graph, split_idx, evaluator)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
  f'Train: {100 * train_acc:.2f}%, '
  f'Valid: {100 * valid_acc:.2f}% '
  f'Test: {100 * test_acc:.2f}%')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Best model: Train: 1.59%, Valid: 1.64% Test: 1.41%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ff505976-6683-4cd6-82f7-fd1506e405f3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>